# Download and Visualize EuroSat Data

In this notebook, we use the torchgeo dataset/datamodules to download the EuroSat dataset (spatial and nonspatial split versions). We then plot a random sample of 6 images (with labels) from the training set and the test set for each dataset version. We also calculate the number of samples in each split for each version (spatial and nonspatial).

In [2]:
import os
from torch.utils.data import DataLoader
from torchgeo.datasets import EuroSAT, EuroSATSpatial, EuroSAT100

### EuroSAT 100

In [14]:
eurosat_100_root = os.path.join("data", "eurosat100")
eurosat_100_dataset = EuroSAT100(eurosat_100_root, download=True)

print(f'Number of images in dataset: {len(eurosat_100_dataset)}')
print(f'Dataset Classes: {eurosat_100_dataset.classes}')

Number of images in dataset: 60
Dataset Classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


### EuroSAT

In [16]:
eurosat_root = os.path.join("data", "eurosat")
eurosat_dataset = EuroSAT(eurosat_root, download=True)

print(f'Number of images in dataset: {len(eurosat_dataset)}')
print(f'Dataset Classes: {eurosat_dataset.classes}')

Number of images in dataset: 16200
Dataset Classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


### EuroSAT Spatial

In [17]:
eurosat_spatial_root = os.path.join("data", "eurosatSpatial")
eurosat_spatial_dataset = EuroSATSpatial(eurosat_spatial_root, download=True)

print(f'Number of images in dataset: {len(eurosat_spatial_dataset)}')
print(f'Dataset Classes: {eurosat_spatial_dataset.classes}')

Number of images in dataset: 16200
Dataset Classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
